In [1]:
import spacy
import requests
import datetime
import json
nlp = spacy.load('en_core_web_lg')

In [2]:
def print_intent(params):
    text = params[0]
    intent = params[1]
    predicate = params[2]
    objects = params[3]
    if not intent:
        print('No intent for "' + text + '", found "' + predicate.text + '" as the root.')
    else:
        print(text,'==>',intent,objects)

    print('-----------------')

In [3]:
test = [
    ("the movie was directed by george clooney","direct"),
    ("movies directed by george clooney","direct"),
    ("movies by george clooney","direct"),
    ("directed by george clooney","direct"),
    ("george clooney director","direct"),
    ("george clooney directing with brad pitt","direct"),
    ("george clooney directed brad pitt","direct"),
    ("george clooney directed by brad pitt","direct"),
    ("The movie stars george clooney","star"),
    ("starring george clooney","star"),
    ("george clooney starred","star"),
    ("george clooney star","star"),
    ("costarring george clooney and brad pitt","costar"),
    ("george clooney and brad pitt","costar"),
    ("george clooney with brad pitt","costar"),
    ("best george clooney movie","best"),
    ("best movie of 2012","best"),
    ("Tom Hanks' best movie","best"),
    ("popular julia roberts","best"),
    ("2012 worst","worst"),
    ("panned julia roberts","worst"),
    ("bad julia roberts","worst")
]

## Let's get the root!

We use the **root** of the sentence/clause as the predicate in the subject-predicate-object structure.

In [4]:
def intent_from_root(text):
    doc = nlp(text)
    intent = None
    predicate = None
    objects = [ent.text for ent in doc.ents]
    
    predicate_taxonomy = {
        'directed':['direct','director','by'],
        'cast':['star','feature','cameo']
    }
        
    #get root of the clause
    for tok in doc:
        if tok.dep_ == "ROOT":
            predicate = tok
    
    #See if the intent is in the taxonomy
    for kind in predicate_taxonomy.keys():
        if predicate.lemma_ in predicate_taxonomy[kind]:
            intent = kind
            
    return [text,intent,predicate,objects]

for t in test:
    print_intent(intent_from_root(t[0]))

the movie was directed by george clooney ==> directed ['george clooney']
-----------------
No intent for "movies directed by george clooney", found "movies" as the root.
-----------------
No intent for "movies by george clooney", found "movies" as the root.
-----------------
directed by george clooney ==> directed ['george clooney']
-----------------
george clooney director ==> directed ['george clooney']
-----------------
No intent for "george clooney directing with brad pitt", found "clooney" as the root.
-----------------
george clooney directed brad pitt ==> directed ['george clooney', 'brad pitt']
-----------------
No intent for "george clooney directed by brad pitt", found "clooney" as the root.
-----------------
The movie stars george clooney ==> cast ['george clooney']
-----------------
starring george clooney ==> cast ['george clooney']
-----------------
george clooney starred ==> cast ['george clooney']
-----------------
george clooney star ==> cast ['george clooney']
-------

In [5]:
def intent_from_root2(text):
    doc = nlp(text)
    intent = None
    predicate = None
    objects = [ent.text for ent in doc.ents]
    
    predicate_taxonomy = {
        'directed':['direct','director'],
        'cast':['star','feature','cameo']
    }
        
    #get root of the clause
    for tok in doc:
        print(tok.text,tok.dep_,tok.pos_)
        if tok.dep_ == "ROOT":
            predicate = tok
    
    print(predicate,objects)
    
    #See if the intent is in the taxonomy
    for kind in predicate_taxonomy.keys():
        if predicate.lemma_ in predicate_taxonomy[kind]:
            intent = kind
    
    if not intent:
        objects[0]
    
    return [text,intent,predicate,objects]
            
for t in test:
    print_intent(intent_from_root(t[0]))

the movie was directed by george clooney ==> directed ['george clooney']
-----------------
No intent for "movies directed by george clooney", found "movies" as the root.
-----------------
No intent for "movies by george clooney", found "movies" as the root.
-----------------
directed by george clooney ==> directed ['george clooney']
-----------------
george clooney director ==> directed ['george clooney']
-----------------
No intent for "george clooney directing with brad pitt", found "clooney" as the root.
-----------------
george clooney directed brad pitt ==> directed ['george clooney', 'brad pitt']
-----------------
No intent for "george clooney directed by brad pitt", found "clooney" as the root.
-----------------
The movie stars george clooney ==> cast ['george clooney']
-----------------
starring george clooney ==> cast ['george clooney']
-----------------
george clooney starred ==> cast ['george clooney']
-----------------
george clooney star ==> cast ['george clooney']
-------